<a href="https://colab.research.google.com/github/ksribhuvana/Desktop_voice_assisstant/blob/master/updated_chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving intents.json to intents.json


In [ ]:
import json

with open('intents.json') as file:
    intents = json.load(file, strict = False) # We don't read the file in strictly so we can use Python regular expressions like '/n' (this is very minor)
intents = intents['intents'] # Get all of the individual intents from our dataset

In [ ]:
print("[", end = "")
for intent in intents:
  print("{", end = "")
  for key, value in intent.items():
    print("{}: {},".format(key, value))
  print("\b\b\n},")
print("\b\b]")

[{tag: greeting,
patterns: ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'],
responses: ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'],
context_set: ,

},
{tag: goodbye,
patterns: ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'],
responses: ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
context_set: ,

},
{tag: age,
patterns: ['how old', 'how old is tim', 'what is your age', 'how old are you', 'age?'],
responses: ['I am 18 years old!', '18 years young!'],
context_set: ,

},
{tag: name,
patterns: ['what is your name', 'what should I call you', 'whats your name?'],
responses: ['You can call me Tim.', "I'm Tim!", "I'm Tim aka Tech With Tim."],
context_set: ,

},
{tag: shop,
patterns: ['Id like to buy something', 'whats on the menu', 'what do you reccommend?', 'could i get something to eat'],
responses: ['We sell chocolate chip cookies for $2!', 'Cookies are on the menu!'],
context_set: ,

},
{tag: hou

In [ ]:
!pip install tensorflow
!pip install tflearn

In [ ]:
!pip uninstall tflearn tensorflow -y
!pip install tflearn tensorflow

  Using cached tflearn-0.5.0-py3-none-any.whl
  Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (601.3 MB)


In [ ]:
import numpy as np
import tensorflow as tf

import tflearn
import random
import pickle

Instructions for updating:
non-resource variables are not supported in the long term


ImportError: cannot import name 'is_sequence' from 'tensorflow.python.util.nest' (/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/nest.py)

In [ ]:
import nltk
nltk.download('all')

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

In [ ]:
retrain_model = True

if retrain_model:
    all_words = []
    all_tags = [] #This will be a list of all the 'tag's associated with the intents
    intent_patterns = [] #This will be a list containing all of the 'patterns' for each intent where each individual pattern is grouped together
    intent_tags = []


    #Here we fill in all of the lists above. Note that we tokenize the UT words in each pattern which means we split each pattern into individual words
    for intent in intents:
        for pattern in intent['patterns']:
            words = nltk.word_tokenize(pattern)

            all_words.extend(words)
            intent_patterns.append(words)
            intent_tags.append(intent['tag'])

        all_tags.append(intent['tag'])

    #Here we stem the words in all_words. This means that we reduce every word down to its root form or stem. This will prevent our chatbot from confusin
    #Very similar words with eachother. For example, the chatbot might normally confuse the words 'running' and 'run' because they appear different even
    #Though they effectively mean the same thing. Stemming will reduce both of these words down to their root form which would be 'run' so the chatbot will
    #No longer be confused
    all_words = [stemmer.stem(word.lower()) for word in all_words]
    all_words = sorted(list(set(all_words)))

    all_tags = sorted(all_tags)

    x_train = []
    y_train = []

    y_empty = [0 for i in range(len(all_tags))]

    #Here we are creating our training set input and output values for our deep learning algorithm
    #We will do this by iterating through our intents and turning each one into a bag of words, or a vector that indicates which words are in each pattern.
    #These bags of words will be the x values and the y values will be the intent that each bag of words is associated with.
    #The machine learning will train on this data and will be able to determine which bag of words its corresponding intent.
    for index, intent in enumerate(intent_patterns):
        bag_of_words = []

        intent_words = [stemmer.stem(word.lower()) for word in intent]

        for word in all_words:
            if word in intent_words:
                bag_of_words.append(1)
            else:
                bag_of_words.append(0)

        one_hot_encode_y = y_empty[:]
        one_hot_encode_y[all_tags.index(intent_tags[index])] = 1

        x_train.append(bag_of_words)
        y_train.append(one_hot_encode_y)

    #Here is the data we will be using to train our neural network later
    x_train = np.array(x_train)
    y_train = np.array(y_train)

    #Here we just save our training data so we don't need to process it again if we just want to run our chatbot
    with open('training_data.pickle', 'wb') as f:
        pickle.dump((all_words, all_tags, x_train, y_train), f)
else:
    with open('training_data.pickle', 'rb') as f:
        all_words, all_tags, x_train, y_train = pickle.load(f)

In [ ]:


neural_net = tflearn.input_data(shape=[None, len(x_train[0])])
neural_net = tflearn.fully_connected(neural_net, 16, activation="ReLU")
neural_net = tflearn.fully_connected(neural_net, 16, activation="ReLU")
neural_net = tflearn.fully_connected(neural_net, len(y_train[0]), activation="softmax")
neural_net = tflearn.regression(neural_net)

model = tflearn.DNN(neural_net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
if retrain_model:
    #Here we train the neural network with the training data we created in the NLP stage

    model.fit(x_train, y_train, n_epoch = 2000, batch_size = 32, show_metric = True)
    model.save('model.tfl')
else:
    model.load('./model.tfl')

Streaming output truncated to the last 5000 lines.
| Adam | epoch: 334 | loss: 0.10817 - acc: 1.0000 -- iter: 26/26
--
Training Step: 335  | total loss: 0.10646 | time: 0.006s
| Adam | epoch: 335 | loss: 0.10646 - acc: 1.0000 -- iter: 26/26
--
Training Step: 336  | total loss: 0.10480 | time: 0.006s
| Adam | epoch: 336 | loss: 0.10480 - acc: 1.0000 -- iter: 26/26
--
Training Step: 337  | total loss: 0.10317 | time: 0.006s
| Adam | epoch: 337 | loss: 0.10317 - acc: 1.0000 -- iter: 26/26
--
Training Step: 338  | total loss: 0.10157 | time: 0.005s
| Adam | epoch: 338 | loss: 0.10157 - acc: 1.0000 -- iter: 26/26
--
Training Step: 339  | total loss: 0.10000 | time: 0.004s
| Adam | epoch: 339 | loss: 0.10000 - acc: 1.0000 -- iter: 26/26
--
Training Step: 340  | total loss: 0.09847 | time: 0.006s
| Adam | epoch: 340 | loss: 0.09847 - acc: 1.0000 -- iter: 26/26
--
Training Step: 341  | total loss: 0.09697 | time: 0.004s
| Adam | epoch: 341 | loss: 0.09697 - acc: 1.0000 -- iter: 26/26
--
Traini

In [ ]:
def text_to_bag(text, all_words):
    #Initialize the bag of words by creating an empty slot for every word in the vector
    bag_of_words = [0 for i in range(len(all_words))]

    #First we split up the input into individual words and stem them so they match the same format as in our vector
    text_words = nltk.word_tokenize(text)
    text_words = [stemmer.stem(word.lower()) for word in text_words]

    #Now we create the bag of words by filling in a 1 for the words that the user used
    for word in text_words:
        if word in all_words:
            bag_of_words[all_words.index(word)] = 1

    #And return the bag of words
    return np.array(bag_of_words)
def chat():
    #Starting message
    print("Enter a message to talk to the bot [type quit to exit].")

    #Reset the context state since there is no context at the beginning of the conversation
    context_state = None

    #This is what the bot will say if it doesn't understand what the user is saying
    default_responses = ['Sorry, Im not sure I know what you mean! You could try rephrasing that or saying something else!',
                         'You confuse me human. Lets talk about something else.',
                         'Im not sure what that means and I dont really care. Lets talk about something else',
                         'I dont understand that! Try rephrasing or saying something else.']

    #This chat loop will go on forever until the user types quit
    while True:
        user_chat = str(input('You: '))
        if user_chat.lower() == 'quit':
            break

        #Convert chat to bag of words
        user_chat_bag = text_to_bag(user_chat, all_words)

        #Pass bag of words into our neural network
        response = model.predict([user_chat_bag])[0]

        #Get the intent that the bag of words is most highly correlated with
        response_index = np.argmax(response)
        response_tag = all_tags[response_index]

        #If the neural network is fairly certain that it has chosen the right intent (and isnt randomly guessing)
        #In this case, we will only get a response if the neural network is more than 80% certain
        if response[response_index] > 0.8:
            for intent in intents:
                #Get the intent that is predicted
                if intent['tag'] == response_tag:
                    #Check if this response is associated with a specific context
                    if 'context_filter' not in intent or 'context_filter' in intent and intent['context_filter'] == context_state:
                        #Get all of the possible responses from this intent
                        possible_responses = intent['responses']
                        #If this intent is associated with a context set, then set the context state
                        if 'context_set' in intent:
                            context_state = intent['context_set']
                        else:
                            context_state = None
                        #Select a random message from the intent responses
                        print(random.choice(possible_responses))
                    else:
                        #Print a did not understand message
                        print(random.choice(default_responses))
        else:
            #Print a did not understand message
            print(random.choice(default_responses))

In [ ]:
chat()

Enter a message to talk to the bot [type quit to exit].
Hello!
Sorry, Im not sure I know what you mean! You could try rephrasing that or saying something else!
We are open 7am-4pm Monday-Friday!
